## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=["EIN","NAME"],inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_type = application_df["APPLICATION_TYPE"].value_counts()
application_type

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_type.loc[lambda x: x < 528].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_ct = application_df['CLASSIFICATION'].value_counts()
classification_ct

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classification_ct.loc[lambda x: x <100].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [8]:
# Convert categorical data to numeric with `pd.get_dummies`
cat_to_num_df = pd.get_dummies(application_df)
cat_to_num_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 50 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   STATUS                        34299 non-null  int64
 1   ASK_AMT                       34299 non-null  int64
 2   IS_SUCCESSFUL                 34299 non-null  int64
 3   APPLICATION_TYPE_Other        34299 non-null  uint8
 4   APPLICATION_TYPE_T10          34299 non-null  uint8
 5   APPLICATION_TYPE_T19          34299 non-null  uint8
 6   APPLICATION_TYPE_T3           34299 non-null  uint8
 7   APPLICATION_TYPE_T4           34299 non-null  uint8
 8   APPLICATION_TYPE_T5           34299 non-null  uint8
 9   APPLICATION_TYPE_T6           34299 non-null  uint8
 10  APPLICATION_TYPE_T7           34299 non-null  uint8
 11  APPLICATION_TYPE_T8           34299 non-null  uint8
 12  AFFILIATION_CompanySponsored  34299 non-null  uint8
 13  AFFILIATION_Family/Parent     3

In [9]:
# Split our preprocessed data into our features and target arrays
y = cat_to_num_df["IS_SUCCESSFUL"]
X = cat_to_num_df.drop(columns="IS_SUCCESSFUL")
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [11]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=49,activation="relu",input_dim=49))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=33,activation="relu"))

# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=14,activation="relu"))

# Fourth hidden layer
nn_model.add(tf.keras.layers.Dense(units=21,activation="relu"))

# Fifth hidden layer
nn_model.add(tf.keras.layers.Dense(units=7,activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 49)                2450      
                                                                 
 dense_1 (Dense)             (None, 33)                1650      
                                                                 
 dense_2 (Dense)             (None, 14)                476       
                                                                 
 dense_3 (Dense)             (None, 21)                315       
                                                                 
 dense_4 (Dense)             (None, 7)                 154       
                                                                 
 dense_5 (Dense)             (None, 1)                 8         
                                                                 
Total params: 5,053
Trainable params: 5,053
Non-trainabl

In [12]:
# Compile the model
nn_model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [13]:
# Train the model
nn_model.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
804/804 [==============================] - 10s 6ms/step - loss: 0.5748 - accuracy: 0.7138
Epoch 2/50
804/804 [==============================] - 7s 8ms/step - loss: 0.5554 - accuracy: 0.7264
Epoch 3/50
804/804 [==============================] - 5s 6ms/step - loss: 0.5525 - accuracy: 0.7284
Epoch 4/50
804/804 [==============================] - 6s 8ms/step - loss: 0.5491 - accuracy: 0.7313
Epoch 5/50
804/804 [==============================] - 5s 7ms/step - loss: 0.5484 - accuracy: 0.7321
Epoch 6/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5465 - accuracy: 0.7326
Epoch 7/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5459 - accuracy: 0.7325
Epoch 8/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5452 - accuracy: 0.7332
Epoch 9/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5444 - accuracy: 0.7334
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5446 - accuracy: 0.734

In [14]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5518 - accuracy: 0.7230 - 557ms/epoch - 2ms/step
Loss: 0.5517654418945312, Accuracy: 0.7230320572853088


In [15]:
# Export our model to HDF5 file
nn_model.save("AlphabetSoupCharity_2.h5")